In [2]:
!pip install jsonpickle

In [0]:
#أستدعاء المكتبات 

import tweepy    #مكتبة تويبي للربط مع تويتر وجمع البيانات
import pandas as pd  #  مكتبة بانداز للتعامل مع الجداول والداتافريم
import re  # هذه المكتبة سوف نستعملها في ازالة الرموز 
import jsonpickle #  لتحويل البيانات التي يتم جلبها من تويتر الى صيغة json 
import string # مكتبة سترينق للتعامل مع النصوص 

In [0]:

# مفاتيح الدخول على تويتر اي بي اي 
CONSUMER_KEY = 'ZSDj9mNN79hLmiflJvz213g1K'
CONSUMER_SECRET = '5UHcS1S3bLdOOWRS0uNUQufiOlbrXxuan8b93iU3gJCHtFdKsE'
ACCESS_TOKEN = '3355371525-wJcqAJVOqbBup6Gz9nXM6Ckx69Uc1xjpUsQpix1'
ACCESS_SECRET = 'B1KBI9ytPN8WBCKtNb7eTO0EToXXPqM3EiRXauX36dLpP'

# Setup access API
# تجهيز او تعريف كود الدخول على تويتر اي بي اي 
# ومن خلاله سوف يتم تمرير مفاتيح الدخول 
def connect_to_twitter_OAuth():
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
    
    api = tweepy.API(auth)
    return api
 
# Create API object
# انشاء اتصال مع تويتر اي بي اي 
api = connect_to_twitter_OAuth()  

In [0]:

# الكلمة او الهاشاتاق الذي سوف يتم البحث عنه
query = 'موبايلي'

# عدد التغريدات التي سوف يتم جلبها 
max_tweets = 300
# لغة التغريدات
lang= 'ar'

#tweetCount = 0

# إنشاء قائمة فارغة وسوف يتم تعبئتها بالتغريدات والمعلومات المهمه لنا 
tweet_list = []

# ارسال طلب لتويتر اي بي اي متضمن كلمة البحث وعدد التغريدات المطلوبة و اللغة 
# وتويتر بدوره سوف يعطينا التغريدات حسب المطلوب
tweets_collected = tweepy.Cursor(api.search,q=query,lang=lang).items(max_tweets)

## تحويل الملف الى صيغة 
## json 
## لكي يتم فيما بعد تحويلها الى جدول 
for tweet in tweets_collected:         

    #Convert to JSON format
    tweettosave = jsonpickle.encode(tweet._json, unpicklable=False).encode('utf-8')
    tweet_list.append(tweettosave)

    #tweetCount += 1

    #Display how many tweets we have collected
    #print("Downloaded {0} tweets".format(tweetCount))

In [0]:
def tweets_to_df(tweets):

    text = []  
    reply = [] 
    retweet = []
    user = []
    screen_name = [] 
    source = []
    in_reply_to_screen_name = [] 
    
    for t in tweets:
        t = jsonpickle.decode(t)
        
        # Text
        text.append(t['text']) 
        
        
            
        
        # Is reply?
        if t['in_reply_to_status_id'] == None:
            reply.append(0)
        else:
            reply.append(1)       
        
        # Retweets count
        retweet.append(t['retweet_count']) 
        
        
        
        # Add user
        user.append(t['user']['name'])

        # Add screen name
        screen_name.append(t['user']['screen_name'])
        # Add sourse
        source.append(t['source'])
        in_reply_to_screen_name.append(t['in_reply_to_screen_name'])
        
    d = {'text': text,
         'is_reply': reply,
         'retweet_count': retweet,
         'user': user,
         'screen_name' : screen_name,
         'source' : source,
         'in_reply_to_screen_name' :in_reply_to_screen_name,
        }
    
    return pd.DataFrame(data = d) 
        


In [0]:
tweets = tweets_to_df(tweet_list)

In [8]:
tweets['text'] = tweets['text'].apply(lambda x: re.sub('[!@#$:).;,?؟&%]', ' ', x.lower()))
tweets['text'] = tweets['text'].apply(lambda x: re.sub('[a-zA-Z]', ' ', x.lower()))

tweets['text'] = tweets['text'].apply(lambda x: re.sub('  ', ' ', x))
tweets['text'][1]

'   09 مرحبا \nتوفر موبايلي خدمة تقسيط الأجهزة لجهاز واحد فقط لعملاء موبايلي للخطوط المفوترة لباقة مفوتر 300 و با…    //  /  4 4  '

In [0]:
tweets = tweets[tweets['source'].str.contains('android')|tweets['source'].str.contains('iphone')|
      tweets['source'].str.contains('iPad')|tweets['source'].str.contains('Mac')|
      tweets['source'].str.contains('Web App')]

In [0]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ؛"-…،'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text

def remove_punctuations(text):
    translator  = re.compile('[%s]' % re.escape(punctuations_list))
    translator .sub(' ', text)
    text = re.sub(' +',' ', text).strip()
    return text

def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1\1', text)


In [0]:
tweets['text'] = tweets['text'].apply(lambda x: normalize_arabic(x))
tweets['text'] = tweets['text'].apply(lambda x: remove_diacritics(x))
tweets['text'] = tweets['text'].apply(lambda x: remove_punctuations(x))
tweets['text'] = tweets['text'].apply(lambda x: remove_repeating_char(x))

In [0]:
tweets['text'].replace(r'…',' ',inplace=True, regex=True)
tweets['text'].replace(r'_',' ',inplace=True, regex=True)
tweets['text'].replace(r'\\\)',' ',inplace=True, regex=True)
tweets['text'].replace(r'،',' ',inplace=True, regex=True)
tweets['text'].replace(r'"',' ',inplace=True, regex=True)
tweets['text'].replace(r'-',' ',inplace=True, regex=True)
tweets['text'].replace(r'؛',' ',inplace=True, regex=True)
tweets['text'].replace(r'/',' ',inplace=True, regex=True)
tweets['text'] = tweets['text'].str.replace('\d+', ' ')
tweets['text'] = tweets['text'].str.replace(' +',' ')

In [0]:
tweets['text'] = tweets['text'].str.replace('\n',' ')

In [17]:
tweets['text']

0          لا الوضع كذا مزري بززيادهه موبايلي الي ايينن 
2                                                موبايلي
3      موبايلي سايلنت و طيران كافر و ملحد 😂✌ عشان بس ...
4      ماهي اخر بطوله حققها الهلال كاس المءسس في اي ع...
5                                     موبايلي طول الوقت✈
                             ...                        
295    موبايلي بيبقي الاربعه وعشرين ساعه كده يااما فا...
296     لاول مره من سنين طويله مع اس تي سي وموبايلي ا...
297       السلام عليكم  عندي شريحه موبايلي بالعرض الق...
298    جوي معطيتني دقيقه و قيقا و سوشال مفتوح بنفس ال...
299    مصوره ب موبايلي ولابس نضارتي وقاعد ع عربيتي  ت...
Name: text, Length: 290, dtype: object

In [0]:
tweets_list  =list(tweets['text'])

In [0]:
tweets_list = list(set(tweets_list))

In [20]:
tweets_list

[' موبايلي مكالمات مفتوحه لجميع الشبكات قيقا بيانات',
 ' بالنهايه موبايلي',
 'تخيلو معاناتي مع شركه موبايلي البيت بدون نت قلنا ماشي  اتفاجا ان حتي مقر عملي في وسط المدينه يومين بدون ',
 'هاي انا كسرت موبايلي بايدي باي',
 'موبايلي عار علي مزودي خدمات الانترنت والله مهما سمعت وقرات عن تظل خدمتهم من افضل الخدمات (صيانه سرعه انترنت تقنيه ',
 'مصوره ب موبايلي ولابس نضارتي وقاعد ع عربيتي  تاخد حياتي تعيشها يابني ',
 ' يمكن انتو موبايلي',
 ' صراحه من اسوا الشركات التي تعاملت معها الاتصالات السعوديه ولقيت كثير من الناس تشتكي وتلومني علي ',
 'بصوا باختصار كده متسترخصش وتجيب موبايل اي كلام عشان انا موبايلي صحيت الصبح لقيته عامل لناس كتير',
 ' اذا زين تدعم اما موبايلي فا لا خلي الانترنت علي زين وابحث في الاعدادات عن خيار ف ',
 'الي يبي اعرضله مقابل موبايلي يجي خاص  ناعم سالب سوالب معصيتي ر حتي',
 ' الله يعافيج موبايلي عطل😭😭',
 '   السلام عليكم  عندي شريحه موبايلي بالعرض القديم خدمه الشراءح المتعدده عليها فاتوره اللي له ',
 'قاتل الله موبايلي',
 'نزولي لوحدي ده مفتقد موبايلي معايا \U0001f97a',
 'ا

In [22]:
!pip install googletrans

  Created wheel for googletrans: filename=googletrans-2.4.0-cp36-none-any.whl size=15777 sha256=b143b7928c3284d9c54dcfd64e6fef4612ccf80509f9b31733d166ca62ffcfad
  Stored in directory: /root/.cache/pip/wheels/50/d6/e7/a8efd5f2427d5eb258070048718fa56ee5ac57fd6f53505f95
Successfully built googletrans


In [0]:
from googletrans import Translator
translator = Translator()

In [32]:
english_tweet = []
arabictweet_2 = []
for arabictweet in tweets_list:
    try:
      translations = translator.translate(arabictweet, dest='en')
    except Exception as e:
        continue
    arabictweet_2.append(translations.origin)
    english_tweet.append(translations.text)
    print(translations.origin, ' -> ', translations.text)

 موبايلي مكالمات مفتوحه لجميع الشبكات قيقا بيانات  ->  Mobily open calls to all networks giga data
 بالنهايه موبايلي  ->  Mobily eventually
تخيلو معاناتي مع شركه موبايلي البيت بدون نت قلنا ماشي  اتفاجا ان حتي مقر عملي في وسط المدينه يومين بدون   ->  Imagine my suffering with Mobily house without net we said Walker surprised that even my headquarters in the center of the city two days without
هاي انا كسرت موبايلي بايدي باي  ->  Hey, I broke the hands of any Mobily
موبايلي عار علي مزودي خدمات الانترنت والله مهما سمعت وقرات عن تظل خدمتهم من افضل الخدمات (صيانه سرعه انترنت تقنيه   ->  Mobily Shame on the Internet service providers and whatever God heard and read about their service remains one of the best services (maintenance speed Internet technique
مصوره ب موبايلي ولابس نضارتي وقاعد ع عربيتي  تاخد حياتي تعيشها يابني   ->  Mobily and cameraman for wearers Ndharta and sitting p Arabiyta my life experienced thing for taking my son
 يمكن انتو موبايلي  ->  Mobily can Anto
 صراحه من اسوا الشر

In [34]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 2.7MB/s 


In [0]:
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [37]:
# --- examples -------
sentences = english_tweet
neg_score =[]
neu_score =[]
pos_score =[]
compound_score =[]
sentment_class = []


analyzer = SentimentIntensityAnalyzer()
for sentence in sentences:
    vs = analyzer.polarity_scores(sentence)
    neg_score.append(vs.get('neg'))
    neu_score.append(vs.get('neu'))
    pos_score.append(vs.get('pos'))
    compound_score.append(vs.get('compound'))
    if vs.get('compound') <= -0.05:
        sentment_class.append(-1)
    elif vs.get('compound') >= 0.05:
        sentment_class.append(1)
    else:
        sentment_class.append(0)

    print(vs.get('compound'))
    print("{:-<65} {}".format(sentence, str(vs)))

0.0
Mobily open calls to all networks giga data---------------------- {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
0.0
Mobily eventually------------------------------------------------ {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
-0.296
Imagine my suffering with Mobily house without net we said Walker surprised that even my headquarters in the center of the city two days without {'neg': 0.111, 'neu': 0.821, 'pos': 0.068, 'compound': -0.296}
-0.4215
Hey, I broke the hands of any Mobily----------------------------- {'neg': 0.318, 'neu': 0.682, 'pos': 0.0, 'compound': -0.4215}
0.4939
Mobily Shame on the Internet service providers and whatever God heard and read about their service remains one of the best services (maintenance speed Internet technique {'neg': 0.096, 'neu': 0.71, 'pos': 0.194, 'compound': 0.4939}
0.0
Mobily and cameraman for wearers Ndharta and sitting p Arabiyta my life experienced thing for taking my son {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0

In [0]:

to_dataframe = {'arabic_tweet': arabictweet_2,
         'translated_tweet': english_tweet,
         'neg': neg_score,
         'neu': neu_score,
         'pos' : pos_score,
         'compound' :compound_score,
         'class' :sentment_class
        }
    
final_data = pd.DataFrame(data = to_dataframe)

In [40]:
final_data.head(50)

,arabic_tweet,translated_tweet,neg,neu,pos,compound,class
0,موبايلي مكالمات مفتوحه لجميع الشبكات قيقا بيانات,Mobily open calls to all networks giga data,0.000,1.000,0.000,0.0000,0
1,بالنهايه موبايلي,Mobily eventually,0.000,1.000,0.000,0.0000,0
2,تخيلو معاناتي مع شركه موبايلي البيت بدون نت قل...,Imagine my suffering with Mobily house without...,0.111,0.821,0.068,-0.2960,-1
3,هاي انا كسرت موبايلي بايدي باي,"Hey, I broke the hands of any Mobily",0.318,0.682,0.000,-0.4215,-1
4,موبايلي عار علي مزودي خدمات الانترنت والله مهم...,Mobily Shame on the Internet service providers...,0.096,0.710,0.194,0.4939,1
5,مصوره ب موبايلي ولابس نضارتي وقاعد ع عربيتي ت...,Mobily and cameraman for wearers Ndharta and s...,0.000,1.000,0.000,0.0000,0
6,يمكن انتو موبايلي,Mobily can Anto,0.000,1.000,0.000,0.0000,0
7,صراحه من اسوا الشركات التي تعاملت معها الاتصا...,EXPRESSLY of the worst companies that dealt wi...,0.360,0.640,0.000,-0.8402,-1
8,بصوا باختصار كده متسترخصش وتجيب موبايل اي كلام...,In short Besoa like that Mtstrechsh and Mobile...,0.000,0.806,0.194,0.5859,1
9,اذا زين تدعم اما موبايلي فا لا خلي الانترنت ع...,If Zain supports either Mobily Fa does not let...,0.000,0.878,0.122,0.3612,1


In [0]:
final_data.to_csv('final_data.csv',encoding='utf-8-sig')